# Pre-Trained BERT Model Load & Fine-tuning with A방송사 datasets
+ 이번 시간에는 A방송사 댓글 데이터를 가지고
+ Pre-Trained BERT Model을 불러온후 Fine-tuning하여 모델링해 보겠습니다.
+ transformers 라이브러리에서 Pre-Trained BERT Model을 제공하고 있으며
+ 우리는 Pre-Trained BERT Model을 불러와 Fine-Tunning해서 
+ A방송사 댓글에 대한 긍정,부정 감성분류하도록 하겠습니다.
+ 참고 사이트
  + https://huggingface.co/
  + https://github.com/huggingface/transformers
  + https://huggingface.co/transformers/custom_datasets.html

### 학습목차
#### A. 필요한 라이브러리 설치 : transformer 
#### B. A방송사 댓글 데이터에 대해 한글 감성분류
1. 라이브러리 임포트
2. 데이터 읽어오기
3. label 컬럼에 대해 LabelEncoding 하기
4. X, y 나누기
5. Train / Test dataset 나누기 (학습 시간 오래 걸려 500개 데이터 사용)
6. Pre-Trained 된 BERT tokenizer 가져오기
7. Pre-Trained 된 BERT tokenizer 사용하여 Train, Test 데이터 토큰화하기
8. Train, Test 데이터셋을 Tensorflow Dataset 형태로 변환
9. Pre-Trained 된 BERT config 확인
10. Pre-Trained 된 BERT 모델 가져오고 컴파일, 학습 수행
11. 학습된 모델로 test_dataset 예측하기
12. 예측 잘 맞는지 확인


## A. 필요한 라이브러리 설치
+ transformers 라이브러리 설치 필요
+ transformers 사용하려면 tensorflow와 keras 2.4.3 이상으로 업그레이드 필요
+ tensorflow 2.8.0 버젼에서는 transformers 일부 모델 학습시 에러 발생하니 주의
+ AIFB 경우 torch 설치 필요

In [1]:
# tensorflow 2.8.0 수행시 에러 발생. 
!pip install transformers  # 4.18.0

### 1. 라이브러리 임포트

## B. A방송사 댓글 데이터에 대해 한글 감성분류

### 1. 라이브러리 임포트

In [2]:
# 경고메세지 끄기
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers

In [5]:
print(transformers.__version__) # 4.18.0
print(tf.__version__) # 2.4.2

4.18.0
2.4.2


### 2. A방송사 댓글 데이터 읽어오기

In [6]:
# pandas read_excel 함수사용하기 위해 xlrd, openpyxl 설치 필요
!pip install openpyxl 

In [7]:
# A_comment_train.xlsx, A_comment_test.xlsx A방송사 댓글 다운로드 및 Pandas read_excel() 함수 활용하여 읽고 합치기

comment_train = pd.read_excel('https://github.com/gzone2000/TEMP_TEST/raw/master/A_comment_train.xlsx', engine='openpyxl')
comment_test = pd.read_excel('https://github.com/gzone2000/TEMP_TEST/raw/master/A_comment_test.xlsx', engine='openpyxl')

comment = pd.concat([comment_train, comment_test])

In [8]:
# 상위 5개 출력
comment.head() 

,Unnamed: 0,data,label
0,0,재미는 있는데 시간이 짧은게 아쉽네요~,긍정
1,1,"OO 관련 내용은 우리 직원과는 거리가 멀었음, 특히, 사내에 홍보할 내용은 아니라고 봄",부정
2,2,스토리가 너무 딱딱해서 별로였음,부정
3,3,프로그램A 화이팅하세요!!,긍정
4,4,높은 곳에 올라가는 모습이 너무 위험해 보여요.,부정


In [9]:
comment.isnull().sum()

Unnamed: 0    0
data          0
label         0
dtype: int64

### 3. label 컬럼에 대해 LabelEncoding 하기

In [10]:
# label 컬럼에 대해 부정 -> 0, 긍정 -> 1 변환

comment['label'] = comment['label'].replace(['부정','긍정'],[0,1])

In [11]:
# 하위 5개 출력
comment.tail() 

,Unnamed: 0,data,label
246,246,영상F서비스로 간편하게 설치!좋아요\n우리 회사화이팅!,1
247,247,모든 업무에서 맡은바 업무에 서 최선을 다하는 모습이 좋습니다! 화이팅 입니다.,1
248,248,"사내방송 특성상 최근 이슈화 되거나, 언급이 자주되는 '키워드'를 중심으로 뉴스를 ...",0
249,249,방송 시간이 너무 길어요.,0
250,250,"처음 들어보는 말들이 많은데,, 설명이 없어서 힘드네요.",0


In [12]:
# comment dataframe 정보 확인
comment.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 502 entries, 0 to 250
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  502 non-null    int64 
 1   data        502 non-null    object
 2   label       502 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 15.7+ KB


### 4. X, y 나누기

In [17]:
# X : data 컬럼과 y : label 컬럼 가져오기
# ndarray로 하면 에러가 나므로 리스트로 만들어야 함.

X = comment.data.to_list()
y = comment.label.to_list()

### 5. Train / Test dataset 나누기

In [18]:
# 데이터를 Train, Test 데이터셋으로 나누기
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2,random_state=48)

In [19]:
# X_train, X_test 사이즈 확인
len(X_train), len(X_test), len(y_train), len(y_test)

(401, 101, 401, 101)

In [20]:
# X_train 데이터 샘플 확인
X_train[:2]

['현재의 구성중에 우리 회사와의 연관성 등을 좀더 고려한 아이템도 추가되면더욱 더 좋은 코너로 자리잡음할 수 있을것같음',
 '무슨 내용인지 파악이 안됩니다. 좀 더 정리가 필요해보여요.']

### 6. Pre-Trained 된 BERT tokenizer 가져오기

In [21]:
# 한글 BERT Model  
bert_model = 'klue/bert-base'

In [22]:
# transformers는 자연어처리를 위해 Pre-Trained BERT, GPT 모델 제공한다.
# transformers의 Config , Tokenizer, SequenceClassification  임포트
# PreTrained 된 "klue/bert-base" BERT 모델의 tokenizer 가져오기

from transformers import AutoConfig, BertTokenizerFast, TFBertForSequenceClassification
tokenizer = BertTokenizerFast.from_pretrained(bert_model)

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [23]:
# tokenizer의 vocab 사이즈 확인
tokenizer.vocab_size

32000

In [17]:
# tokenizer의 vocab 리스트
tokenizer.vocab

{'음직': 24131,
 '232': 25338,
 '왕좌': 30795,
 '테르': 22168,
 '오바': 6353,
 '공경': 22056,
 '고마': 6914,
 '애인': 9757,
 '車': 503,
 '추억': 6469,
 '올스타': 14213,
 '모빌리티': 30397,
 '교육청': 5220,
 '사용료': 16440,
 '3700': 29333,
 '콤': 1730,
 '나일': 26189,
 '서둘러': 8468,
 '홍보': 4765,
 '토르': 24321,
 '##소사': 30383,
 '황새': 27422,
 '이어진다': 10231,
 '적셔': 30848,
 '##멜': 2683,
 '헤아리': 20086,
 '물려주': 20158,
 '임대료': 10135,
 '수장': 10357,
 '바게트': 27618,
 '정직': 10089,
 '##ne': 10933,
 '비대': 7751,
 '된답니다': 28153,
 '옷감': 25830,
 '던스': 28470,
 '자문': 6842,
 '총탄': 27142,
 '느리': 13704,
 '복원사업': 30093,
 '##쓸이': 21654,
 '지만은': 15027,
 '##쏟': 3244,
 '하지원': 29346,
 '신안': 14946,
 '기부자': 26436,
 '몇몇': 7396,
 '진에어': 23995,
 '한쪽': 6986,
 '이승철': 30482,
 '고층': 18093,
 '##보내': 14179,
 '세대': 4489,
 '난립': 23126,
 '서툴': 24255,
 '끌어올린': 27185,
 '영영': 22672,
 '침울': 29100,
 '명화': 29257,
 'Kar': 30317,
 '기아': 6194,
 '자질': 9955,
 '부추기': 15155,
 '수명': 9038,
 '기념일': 15781,
 '##eak': 31146,
 '금지': 5040,
 '류현진': 8659,
 '포착': 8871,
 '##경': 2382,
 '

### 7. Pre-Trained 된 BERT tokenizer 사용하여 Train, Test 데이터 토큰화하기

In [24]:
# X_train, X_test 데이터를 토큰 > 정수ids > padding  수행

train_encodings = tokenizer(X_train, truncation=True, padding=True, )
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [28]:
# train input_ids : 문장을 숫자 매핑하는 keras Tokenizer의 texts_to_sequences + pad_sequences 후의 결과값
print(train_encodings['input_ids'][0])

[2, 3738, 2079, 3896, 2284, 2170, 3616, 3769, 2522, 2079, 6597, 2047, 886, 2069, 1556, 2320, 4146, 2470, 6226, 2119, 4140, 2496, 2460, 2320, 2682, 831, 1560, 2073, 7849, 2200, 3798, 2741, 2053, 2085, 1295, 1513, 2069, 2728, 2246, 2053, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [30]:
# test input_ids
print(test_encodings['input_ids'][0])

[2, 9875, 2205, 3011, 10283, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


### 8. Train, Test 데이터셋을 Tensorflow Dataset 형태로 변환

In [31]:
# 반드시 입력할 데이터를 tensorflow Dataset 형태로 만들어야 함

train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train))
train_dataset = train_dataset.shuffle(1000).batch(16).cache().prefetch(tf.data.experimental.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test))
test_dataset = test_dataset.batch(16).cache().prefetch(tf.data.experimental.AUTOTUNE)

### 9. Pre-Trained 된 BERT config 확인

In [32]:
# klue/bert-base BERT config 확인
# architectures : BertForMaskedLM

config = AutoConfig.from_pretrained(bert_model)
config

BertConfig {
  "_name_or_path": "klue/bert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

### 10. Pre-Trained 된 BERT 모델 가져오고 컴파일, 학습 수행
+ 모델 인자로 num_labels=2 입력
+ Pytorch로 학습된 모델이기 때문에 Tensorflow에서 사용하기 위해서는 from_pretrained() 인자에 from_pt=True를 넣어줌으로써 Tensorflow모델로 변환 및 로드 할 수 있다.


In [33]:
# 모델 입력으로  몇개로 분류할지 알려주는 num_labels 인자와 
# from_pt=True 으로 pytorch 학습된 정보를 tensorflow로 불러올수 있도록 한다.

from transformers import TFBertForSequenceClassification
model = TFBertForSequenceClassification.from_pretrained(bert_model, num_labels=2, from_pt=True)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
model.fit(train_dataset, epochs=1, batch_size=16, validation_data=(test_dataset))

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


26/26 [==============================] - 25s 420ms/step - loss: 0.3429 - accuracy: 0.8418 - val_loss: 0.0122 - val_accuracy: 1.0000


In [34]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  110617344 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 110,618,882
Trainable params: 110,618,882
Non-trainable params: 0
_________________________________________________________________


### 11. 학습된 모델로 test_dataset 예측하기

In [35]:
y_test_pred = model.predict(test_dataset)

In [36]:
y_test_pred.logits.shape

(101, 2)

In [37]:
y_test_pred.logits[:10]

array([[-2.456198 ,  2.408847 ],
       [-2.4773054,  2.8036425],
       [-2.5904021,  2.7384388],
       [-1.6775036,  1.6229781],
       [ 2.8301947, -2.9206688],
       [-2.416474 ,  2.6521437],
       [ 2.4268982, -2.5700731],
       [-2.6145139,  2.6411088],
       [-2.2296927,  2.1927934],
       [-2.641478 ,  2.6470482]], dtype=float32)

### 12. 예측 잘 맞는지 확인

In [38]:
# 예측 결과를 DataFrame 넣기

df = pd.DataFrame(np.argmax(y_test_pred.logits, axis=1), columns=['predict'])
df

,predict
0,1
1,1
2,1
3,1
4,0
...,...
96,0
97,1
98,0
99,1


In [39]:
# 정답도 DataFrame에 넣기

df['true'] = y_test
df

,predict,true
0,1,1
1,1,1
2,1,1
3,1,1
4,0,0
...,...,...
96,0,0
97,1,1
98,0,0
99,1,1


In [40]:
np.sum(df['true'] == df['predict'])/len(df)

1.0

# 배운 내용 정리
1. transformers는 자연어처리를 위해 Pre-Trained BERT, GPT 모델 제공합니다.
2. transformers의 Config , Tokenizer,  SequenceClassification 불러와서
3. BERT Tokenizer로 문장들을 토큰화하고 Tensorflow Dataset 포맷으로 만든후
4. Pre-Trained BERT Model을 가져와 우리 입맛에 맞게 fine-Tunning해서 A방송사 댓글 감성분류해 보았습니다.
5. 필요시 Pre-Trained BERT Model 가져와 Fine-Tunning해서 자연어 처리에 활용하시면 되겠습니다.